In [20]:
import cv2
import mediapipe as mp
import numpy as np
import csv
import os

# Inicializar MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=False,
    max_num_faces=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# --- Mapeo de emociones a clases numéricas ---
CLASES = {
    'Neutro': 0,
    'Feliz': 1,
    'Triste': 2,
    'Enojado': 3,
    'Sorprendido': 4
}

# --- Archivo CSV ---
csv_filename = "emociones.csv"
header = [
    'ratio_boca', 'ratio_sonrisa', 'ratio_curvatura', 'ratio_cejas', 
    'ratio_ojos', 'ratio_aspecto_ojos', 'ratio_tension_parpados', 
    'ratio_grosor_labios', 'ratio_compresion_boca', 
    'ratio_nariz_boca', 'ratio_dist_cejas', 'clase'
]


# Puntos clave para detección de emociones
PUNTOS_CLAVE = {
    # Cejas
    'ceja_izq_interior': 70,
    'ceja_izq_exterior': 107,
    'ceja_der_interior': 300,
    'ceja_der_exterior': 336,
    'ceja_izq_centro': 105,
    'ceja_der_centro': 334,
    
    # Ojos - AMPLIADO para mejor detección
    'ojo_izq_superior': 159,
    'ojo_izq_inferior': 145,
    'ojo_der_superior': 386,
    'ojo_der_inferior': 374,
    'ojo_izq_exterior': 33,
    'ojo_der_exterior': 263,
    'ojo_izq_interior': 133,
    'ojo_der_interior': 362,
    # Párpados superiores (más puntos)
    'parpado_izq_medio': 160,
    'parpado_der_medio': 387,
    # Parte inferior del ojo (bolsas)
    'ojo_izq_inf_medio': 144,
    'ojo_der_inf_medio': 373,
    
    # Boca - AMPLIADO
    'boca_superior': 13,
    'boca_inferior': 14,
    'comisura_izq': 61,
    'comisura_der': 291,
    'boca_izq': 78,
    'boca_der': 308,
    'labio_sup_centro': 0,
    'labio_inf_centro': 17,
    # Bordes de labios
    'labio_sup_izq_ext': 185,
    'labio_sup_der_ext': 409,
    'labio_inf_izq_ext': 62,
    'labio_inf_der_ext': 292,
    # Centro de labios (para detectar fruncimiento)
    'labio_sup_interno': 12,
    'labio_inf_interno': 15,

    # Referencias
    'nariz_punta': 1,
    'nariz_puente': 168,
    'barbilla': 152,
    'nariz_lado_izq': 48,
    'nariz_lado_der': 278,
}

# Crear CSV con encabezado si no existe
if not os.path.exists(csv_filename):
    with open(csv_filename, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(header)

def calcular_distancia(p1, p2):
    """Calcula la distancia euclidiana entre dos puntos"""
    return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

def extraer_coordenadas(landmarks, w, h, indices):
    """Extrae coordenadas de puntos específicos"""
    coords = {}
    for nombre, idx in indices.items():
        landmark = landmarks.landmark[idx]
        coords[nombre] = (int(landmark.x * w), int(landmark.y * h))
    return coords

def detectar_emocion(coords):
    """
    Detecta la emoción basándose en ratios de distancias faciales
    ENFOQUE PRINCIPAL EN OJOS ENTRECERRADOS Y BOCA FRUNCIDA PARA ENOJO
    """
    # Distancia de referencia
    dist_referencia = calcular_distancia(coords['nariz_puente'], coords['barbilla'])
    
    if dist_referencia == 0:
        return 'Neutro', 0, {}
    
    # ============ MEDIDAS BÁSICAS ============
    
    # Apertura de boca (vertical)
    apertura_boca = calcular_distancia(coords['boca_superior'], coords['boca_inferior'])
    ratio_boca = apertura_boca / dist_referencia
    
    # Ancho de la boca
    ancho_boca_actual = calcular_distancia(coords['comisura_izq'], coords['comisura_der'])
    ancho_boca_neutral = calcular_distancia(coords['boca_izq'], coords['boca_der'])
    ratio_sonrisa = ancho_boca_actual / ancho_boca_neutral if ancho_boca_neutral > 0 else 1.0
    
    # Curvatura de boca
    centro_boca_y = (coords['boca_superior'][1] + coords['boca_inferior'][1]) / 2
    comisura_izq_y = coords['comisura_izq'][1]
    comisura_der_y = coords['comisura_der'][1]
    desviacion_comisuras = ((comisura_izq_y - centro_boca_y) + (comisura_der_y - centro_boca_y)) / 2
    ratio_curvatura = desviacion_comisuras / dist_referencia
    
    # Altura de cejas
    altura_ceja_izq = calcular_distancia(coords['ceja_izq_centro'], coords['ojo_izq_superior'])
    altura_ceja_der = calcular_distancia(coords['ceja_der_centro'], coords['ojo_der_superior'])
    altura_cejas_promedio = (altura_ceja_izq + altura_ceja_der) / 2
    ratio_cejas = altura_cejas_promedio / dist_referencia
    
    # ============ MEDIDAS CRÍTICAS PARA ENOJO ============
    
    # 1. OJOS ENTRECERRADOS (CRÍTICO) - Apertura vertical reducida
    apertura_ojo_izq = calcular_distancia(coords['ojo_izq_superior'], coords['ojo_izq_inferior'])
    apertura_ojo_der = calcular_distancia(coords['ojo_der_superior'], coords['ojo_der_inferior'])
    apertura_ojos_promedio = (apertura_ojo_izq + apertura_ojo_der) / 2
    ratio_ojos = apertura_ojos_promedio / dist_referencia
    
    # 2. RATIO DE ASPECTO DEL OJO (ancho vs alto) - Ojos más "estrechos"
    ancho_ojo_izq = calcular_distancia(coords['ojo_izq_interior'], coords['ojo_izq_exterior'])
    ancho_ojo_der = calcular_distancia(coords['ojo_der_interior'], coords['ojo_der_exterior'])
    ancho_ojos_promedio = (ancho_ojo_izq + ancho_ojo_der) / 2
    
    # Ratio aspecto ojo (ancho/alto) - Aumenta cuando se entrecierran
    ratio_aspecto_ojo_izq = ancho_ojo_izq / (apertura_ojo_izq + 0.001)
    ratio_aspecto_ojo_der = ancho_ojo_der / (apertura_ojo_der + 0.001)
    ratio_aspecto_ojos = (ratio_aspecto_ojo_izq + ratio_aspecto_ojo_der) / 2
    
    # 3. TENSIÓN PÁRPADOS - Párpado inferior sube cuando entrecierras
    # Medimos la distancia del párpado inferior a la línea media del ojo
    centro_ojo_izq_y = (coords['ojo_izq_superior'][1] + coords['ojo_izq_inferior'][1]) / 2
    centro_ojo_der_y = (coords['ojo_der_superior'][1] + coords['ojo_der_inferior'][1]) / 2
    
    # Qué tan arriba está el párpado inferior del centro
    tension_parpado_izq = centro_ojo_izq_y - coords['ojo_izq_inferior'][1]
    tension_parpado_der = centro_ojo_der_y - coords['ojo_der_inferior'][1]
    ratio_tension_parpados = ((tension_parpado_izq + tension_parpado_der) / 2) / dist_referencia
    
    # 4. BOCA FRUNCIDA (CRÍTICO) - Labios hacia adentro, boca pequeña
    # Distancia entre los labios superior e inferior (grosor visible)
    grosor_labio_sup = calcular_distancia(coords['labio_sup_centro'], coords['labio_sup_interno'])
    grosor_labio_inf = calcular_distancia(coords['labio_inf_centro'], coords['labio_inf_interno'])
    ratio_grosor_labios = (grosor_labio_sup + grosor_labio_inf) / dist_referencia
    
    # 5. COMPRESIÓN HORIZONTAL DE LA BOCA - Boca más estrecha al fruncir
    ancho_boca_externa = calcular_distancia(coords['labio_sup_izq_ext'], coords['labio_sup_der_ext'])
    ratio_compresion_boca = ancho_boca_externa / dist_referencia
    
    # 6. RELACIÓN ANCHO/APERTURA DE BOCA - Boca cerrada pero no necesariamente ancha
    if apertura_boca > 0:
        ratio_forma_boca = ancho_boca_actual / (apertura_boca + 0.001)
    else:
        ratio_forma_boca = 10.0  # Valor alto si está cerrada
    
    # 7. NARIZ ARRUGADA (secundario) - Lados de nariz suben ligeramente
    dist_nariz_boca_izq = calcular_distancia(coords['nariz_lado_izq'], coords['comisura_izq'])
    dist_nariz_boca_der = calcular_distancia(coords['nariz_lado_der'], coords['comisura_der'])
    ratio_nariz_boca = ((dist_nariz_boca_izq + dist_nariz_boca_der) / 2) / dist_referencia
    
    # Distancia entre cejas (secundario ahora)
    dist_cejas = calcular_distancia(coords['ceja_izq_interior'], coords['ceja_der_interior'])
    ratio_dist_cejas = dist_cejas / dist_referencia

    # Ratios info
    ratios_info = {
        'ratio_boca': ratio_boca,
        'ratio_sonrisa': ratio_sonrisa,
        'ratio_curvatura': ratio_curvatura,
        'ratio_cejas': ratio_cejas,
        'ratio_ojos': ratio_ojos,
        'ratio_aspecto_ojos': ratio_aspecto_ojos,
        'ratio_tension_parpados': ratio_tension_parpados,
        'ratio_grosor_labios': ratio_grosor_labios,
        'ratio_compresion_boca': ratio_compresion_boca,
        'ratio_nariz_boca': ratio_nariz_boca,
        'ratio_dist_cejas': ratio_dist_cejas,
    }
    
    scores = {
        'Feliz': 0,
        'Triste': 0,
        'Enojado': 0,
        'Sorprendido': 0,
        'Neutro': 5  
    }
    
    # ===== FELICIDAD =====
    if ratio_sonrisa > 1.20:
        scores['Feliz'] += 4
        scores['Neutro'] -= 3
    elif ratio_sonrisa > 1.12:
        scores['Feliz'] += 2
        scores['Neutro'] -= 1
    
    if ratio_curvatura < -0.015:
        scores['Feliz'] += 4
        scores['Neutro'] -= 3
    elif ratio_curvatura < -0.008:
        scores['Feliz'] += 2
        scores['Neutro'] -= 1
    
    if 0.03 < ratio_boca < 0.10 and ratio_sonrisa > 1.15:
        scores['Feliz'] += 2
    
    # ===== TRISTEZA =====
    if ratio_curvatura > 0.012:
        scores['Triste'] += 5
        scores['Neutro'] -= 3
    elif ratio_curvatura > 0.006:
        scores['Triste'] += 3
        scores['Neutro'] -= 2
    
    if ratio_sonrisa < 1.05:
        scores['Triste'] += 2
    
    if ratio_boca < 0.025:
        scores['Triste'] += 1
    
    # ===== ENOJO (REDISEÑADO - ENFOQUE EN OJOS Y BOCA) =====
    puntos_enojo = 0
    
    # CRITERIO 1: OJOS ENTRECERRADOS (MUY IMPORTANTE)
    if ratio_ojos < 0.055:  # Ojos MUY entrecerrados
        scores['Enojado'] += 8
        puntos_enojo += 3
        scores['Neutro'] -= 4
    elif ratio_ojos < 0.068:  # Ojos moderadamente entrecerrados
        scores['Enojado'] += 5
        puntos_enojo += 2
        scores['Neutro'] -= 2
    elif ratio_ojos < 0.078:  # Ojos ligeramente entrecerrados
        scores['Enojado'] += 3
        puntos_enojo += 1
    
    # CRITERIO 2: RATIO DE ASPECTO (ojos más alargados/estrechos)
    if ratio_aspecto_ojos > 5.5:  # Ojos muy alargados
        scores['Enojado'] += 6
        puntos_enojo += 2
    elif ratio_aspecto_ojos > 4.8:  # Ojos alargados
        scores['Enojado'] += 4
        puntos_enojo += 1
    # CRITERIO 3: TENSIÓN EN PÁRPADOS (párpado inferior sube)
    if ratio_tension_parpados < -0.008:  # Párpados muy tensos
        scores['Enojado'] += 5
        puntos_enojo += 1.5
    elif ratio_tension_parpados < -0.004:
        scores['Enojado'] += 3
        puntos_enojo += 0.5
    
    # CRITERIO 4: BOCA FRUNCIDA - Labios delgados (hacia adentro)
    if ratio_grosor_labios < 0.012:  # Labios MUY hacia adentro
        scores['Enojado'] += 7
        puntos_enojo += 2.5
        scores['Neutro'] -= 3
    elif ratio_grosor_labios < 0.018:  # Labios moderadamente hacia adentro
        scores['Enojado'] += 5
        puntos_enojo += 1.5
        scores['Neutro'] -= 2
    elif ratio_grosor_labios < 0.025:  # Labios ligeramente hacia adentro
        scores['Enojado'] += 3
        puntos_enojo += 0.5
    
    # CRITERIO 5: BOCA COMPRIMIDA HORIZONTALMENTE
    if ratio_compresion_boca < 0.30:  # Boca muy estrecha
        scores['Enojado'] += 5
        puntos_enojo += 1.5
    elif ratio_compresion_boca < 0.35:  # Boca estrecha
        scores['Enojado'] += 3
        puntos_enojo += 1
    
    # CRITERIO 6: BOCA CERRADA con tensión
    if ratio_boca < 0.025 and ratio_sonrisa < 1.08:  # Boca cerrada y tensa
        scores['Enojado'] += 4
        puntos_enojo += 1
    elif ratio_boca < 0.035 and ratio_sonrisa < 1.10:
        scores['Enojado'] += 2
        puntos_enojo += 0.5
    
    # CRITERIO 7: Cejas juntas (SECUNDARIO ahora)
    if ratio_dist_cejas < 0.28:
        scores['Enojado'] += 3
        puntos_enojo += 0.5
    
    # CRITERIO 8: Nariz arrugada
    if ratio_nariz_boca < 0.22:
        scores['Enojado'] += 2
        puntos_enojo += 0.5

    
    # PENALIZACIONES
    if ratio_sonrisa > 1.15:  # Está sonriendo
        scores['Enojado'] = 0
        puntos_enojo = 0
    
    if ratio_curvatura < -0.010:  # Comisuras hacia arriba
        scores['Enojado'] = max(0, scores['Enojado'] - 6)
        puntos_enojo = max(0, puntos_enojo - 2)
    
    if ratio_ojos > 0.095:  # Ojos muy abiertos (incompatible)
        scores['Enojado'] = max(0, scores['Enojado'] - 5)
    
    # Requiere al menos 2.5 puntos para ser enojado
    if puntos_enojo < 2.5:
        scores['Enojado'] = 0
    
    # ===== SORPRESA =====
    condiciones_sorpresa = 0
    
    if ratio_cejas > 0.18:
        scores['Sorprendido'] += 4
        condiciones_sorpresa += 1
    
    if ratio_ojos > 0.09:
        scores['Sorprendido'] += 4
        condiciones_sorpresa += 1
    
    if ratio_boca > 0.15:
        scores['Sorprendido'] += 4
        condiciones_sorpresa += 1
    
    if condiciones_sorpresa < 2:
        scores['Sorprendido'] = 0
    else:
        scores['Neutro'] -= 5
    
    # Asegurar que los scores no sean negativos
    for emocion in scores:
        scores[emocion] = max(0, scores[emocion])
    
    # Seleccionar emoción con mayor score
    emocion = max(scores, key=scores.get)
    confianza = scores[emocion]
    
    if confianza < 3:
        emocion = 'Neutro'
        confianza = 5
    
    return emocion, confianza, ratios_info

# Configuración de colores por emoción
COLORES_EMOCION = {
    'Feliz': (0, 255, 0),
    'Triste': (255, 0, 0),
    'Enojado': (0, 0, 255),
    'Sorprendido': (0, 255, 255),
    'Neutro': (200, 200, 200)
}

# Captura de video
cap = cv2.VideoCapture(2)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)
    
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            h, w, _ = frame.shape
            
            coords = extraer_coordenadas(face_landmarks, w, h, PUNTOS_CLAVE)
            emocion, confianza, ratios_info = detectar_emocion(coords)
            
            color = COLORES_EMOCION[emocion]
            
            # Dibujar puntos - RESALTAR ojos y boca
            for nombre, (x, y) in coords.items():
                if 'ojo' in nombre or 'parpado' in nombre:
                    cv2.circle(frame, (x, y), 3, (255, 255, 0), -1)  # Amarillo para ojos
                elif 'labio' in nombre or 'boca' in nombre:
                    cv2.circle(frame, (x, y), 3, (255, 0, 255), -1)  # Magenta para boca
                else:
                    cv2.circle(frame, (x, y), 2, (200, 200, 200), -1)
            
            # Mostrar emoción
            texto_emocion = f" {emocion} "
            font = cv2.FONT_HERSHEY_SIMPLEX
            (text_w, text_h), _ = cv2.getTextSize(texto_emocion, font, 1.2, 3)
            cv2.rectangle(frame, (5, 5), (15 + text_w, 45), color, -1)
            cv2.putText(frame, texto_emocion, (10, 35), 
                       font, 1.2, (0, 0, 0), 3)

             # === Guardar ratios y clase en CSV ===
            fila = [ratios_info[key] for key in header[:-1]]  # Todos los ratios
            fila.append(CLASES.get(emocion, 0))  # Clase numérica
            with open(csv_filename, 'a', newline='') as f:
                writer = csv.writer(f)
                writer.writerow(fila)
            
    
    cv2.imshow('Detector de Emociones', frame)
    
    key = cv2.waitKey(1)
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Código lector del decision tree

In [26]:
import cv2
import mediapipe as mp
import numpy as np
import joblib
import os
import pandas as pd

# Inicializar FaceMesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=False,
    max_num_faces=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# Cargar el modelo entrenado
modelo = joblib.load("modelo_emociones.pkl")

# Las mismas features que usaste para entrenar (orden idéntico al CSV)
FEATURES = [
    'ratio_boca', 'ratio_sonrisa', 'ratio_curvatura', 'ratio_cejas', 
    'ratio_ojos', 'ratio_aspecto_ojos', 'ratio_tension_parpados', 
    'ratio_grosor_labios', 'ratio_compresion_boca', 
    'ratio_nariz_boca', 'ratio_dist_cejas'
]

# Mapeo de clases
CLASES_INV = {
    0: 'Neutro',
    1: 'Feliz',
    2: 'Triste',
    3: 'Enojado',
    4: 'Sorprendido'
}

# Colores para mostrar
COLORES_EMOCION = {
    'Feliz': (0, 255, 0),
    'Triste': (255, 0, 0),
    'Enojado': (0, 0, 255),
    'Sorprendido': (0, 255, 255),
    'Neutro': (200, 200, 200)
}

# ===== FUNCIONES =====
def calcular_distancia(p1, p2):
    return np.sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)

def extraer_coordenadas(landmarks, w, h, indices):
    coords = {}
    for nombre, idx in indices.items():
        lm = landmarks.landmark[idx]
        coords[nombre] = (int(lm.x * w), int(lm.y * h))
    return coords

# Puntos clave para detección de emociones
PUNTOS_CLAVE = {
    # Cejas
    'ceja_izq_interior': 70,
    'ceja_izq_exterior': 107,
    'ceja_der_interior': 300,
    'ceja_der_exterior': 336,
    'ceja_izq_centro': 105,
    'ceja_der_centro': 334,
    
    # Ojos - AMPLIADO para mejor detección
    'ojo_izq_superior': 159,
    'ojo_izq_inferior': 145,
    'ojo_der_superior': 386,
    'ojo_der_inferior': 374,
    'ojo_izq_exterior': 33,
    'ojo_der_exterior': 263,
    'ojo_izq_interior': 133,
    'ojo_der_interior': 362,
    # Párpados superiores (más puntos)
    'parpado_izq_medio': 160,
    'parpado_der_medio': 387,
    # Parte inferior del ojo (bolsas)
    'ojo_izq_inf_medio': 144,
    'ojo_der_inf_medio': 373,
    
    # Boca - AMPLIADO
    'boca_superior': 13,
    'boca_inferior': 14,
    'comisura_izq': 61,
    'comisura_der': 291,
    'boca_izq': 78,
    'boca_der': 308,
    'labio_sup_centro': 0,
    'labio_inf_centro': 17,
    # Bordes de labios
    'labio_sup_izq_ext': 185,
    'labio_sup_der_ext': 409,
    'labio_inf_izq_ext': 62,
    'labio_inf_der_ext': 292,
    # Centro de labios (para detectar fruncimiento)
    'labio_sup_interno': 12,
    'labio_inf_interno': 15,

    # Referencias
    'nariz_punta': 1,
    'nariz_puente': 168,
    'barbilla': 152,
    'nariz_lado_izq': 48,
    'nariz_lado_der': 278,
}

def calcular_ratios(coords):
    """ Calcula los mismos ratios que usabas en el script original """
    dist_ref = calcular_distancia(coords['nariz_puente'], coords['barbilla'])
    if dist_ref == 0:
        return {key: 0 for key in FEATURES}

    # === Calcula tus ratios igual que en tu script anterior ===
    ratio_boca = calcular_distancia(coords['boca_superior'], coords['boca_inferior']) / dist_ref
    ratio_sonrisa = calcular_distancia(coords['comisura_izq'], coords['comisura_der']) / dist_ref
    ratio_curvatura = ((coords['comisura_izq'][1] + coords['comisura_der'][1]) / 2 - 
                       (coords['boca_superior'][1] + coords['boca_inferior'][1]) / 2) / dist_ref
    ratio_cejas = (
        (calcular_distancia(coords['ceja_izq_centro'], coords['ojo_izq_superior']) +
         calcular_distancia(coords['ceja_der_centro'], coords['ojo_der_superior'])) / 2
    ) / dist_ref
    ratio_ojos = (
        (calcular_distancia(coords['ojo_izq_superior'], coords['ojo_izq_inferior']) +
         calcular_distancia(coords['ojo_der_superior'], coords['ojo_der_inferior'])) / 2
    ) / dist_ref
    ratio_aspecto_ojos = (
        (calcular_distancia(coords['ojo_izq_interior'], coords['ojo_izq_exterior']) /
         (calcular_distancia(coords['ojo_izq_superior'], coords['ojo_izq_inferior']) + 0.001) +
         calcular_distancia(coords['ojo_der_interior'], coords['ojo_der_exterior']) /
         (calcular_distancia(coords['ojo_der_superior'], coords['ojo_der_inferior']) + 0.001)
        ) / 2
    )
    ratio_tension_parpados = (((coords['ojo_izq_superior'][1] + coords['ojo_izq_inferior'][1]) / 2 -
                               coords['ojo_izq_inferior'][1]) / dist_ref)
    ratio_grosor_labios = (
        calcular_distancia(coords['labio_sup_centro'], coords['labio_sup_interno']) +
        calcular_distancia(coords['labio_inf_centro'], coords['labio_inf_interno'])
    ) / dist_ref
    ratio_compresion_boca = calcular_distancia(coords['labio_sup_izq_ext'], coords['labio_sup_der_ext']) / dist_ref
    ratio_nariz_boca = (
        (calcular_distancia(coords['nariz_lado_izq'], coords['comisura_izq']) +
         calcular_distancia(coords['nariz_lado_der'], coords['comisura_der'])) / 2
    ) / dist_ref
    ratio_dist_cejas = calcular_distancia(coords['ceja_izq_interior'], coords['ceja_der_interior']) / dist_ref

    return {
        'ratio_boca': ratio_boca,
        'ratio_sonrisa': ratio_sonrisa,
        'ratio_curvatura': ratio_curvatura,
        'ratio_cejas': ratio_cejas,
        'ratio_ojos': ratio_ojos,
        'ratio_aspecto_ojos': ratio_aspecto_ojos,
        'ratio_tension_parpados': ratio_tension_parpados,
        'ratio_grosor_labios': ratio_grosor_labios,
        'ratio_compresion_boca': ratio_compresion_boca,
        'ratio_nariz_boca': ratio_nariz_boca,
        'ratio_dist_cejas': ratio_dist_cejas
    }

# ======== DETECTOR EN TIEMPO REAL ========
cap = cv2.VideoCapture(2)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb)
    
    if results.multi_face_landmarks:
        for lm in results.multi_face_landmarks:
            h, w, _ = frame.shape
            coords = extraer_coordenadas(lm, w, h, PUNTOS_CLAVE)
            ratios = calcular_ratios(coords)

            # Convertir en lista en el mismo orden que las columnas
            X_pred = pd.DataFrame([[ratios[f] for f in FEATURES]], columns=FEATURES)
            clase_pred = int(modelo.predict(X_pred)[0])
            emocion = CLASES_INV[clase_pred]
            color = COLORES_EMOCION[emocion]

            cv2.rectangle(frame, (5, 5), (220, 55), color, -1)
            cv2.putText(frame, emocion, (15, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 0), 3)
    
    cv2.imshow("Detección con Decision Tree", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
